In [1]:
from codes.downloader import create_local_data, download_source_data
from codes.parameters import categories
category = categories[0] # "All_Beauty"
# download_source_data(category)
# create_local_data(category)

In [2]:
from codes.filter import filter
data = filter(category = category, min_length = 3, min_reviews_per_product = 5, min_average_rating = 1)

In [3]:
sample = data[['text','rating']].head(10000)
sample

,text,rating
0,This spray is really nice. It smells really go...,5.0
1,"This product does what I need it to do, I just...",4.0
2,"Smells good, feels great!",5.0
5,The polish was quiet thick and did not apply s...,4.0
7,These were lightweight and soft but much too s...,3.0
...,...,...
13312,This is a cute headband and is very comfortabl...,2.0
13313,This is an honest non paid for non free item g...,4.0
13315,Well made and good assortment.,5.0
13316,"I’m a barber here throwing my two cents, just ...",5.0


In [7]:
texts = list(sample.text)
labels = [int(x)-1 for x in sample.rating]

In [11]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch

train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenizer(train_texts, padding="max_length", truncation=True, max_length=128)
val_encodings = tokenizer(val_texts, padding="max_length", truncation=True, max_length=128)


def prepare_dataset(encodings, labels):
    return Dataset.from_dict({
        'input_ids': encodings['input_ids'],
        'attention_mask': encodings['attention_mask'],
        'labels': labels
    })

train_dataset = prepare_dataset(train_encodings, train_labels)
val_dataset = prepare_dataset(val_encodings, val_labels)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)
#model.to(torch.device("cuda")) 

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).to(torch.device("cuda"))
    accuracy = (predictions == torch.tensor(labels).to(torch.device("cuda"))).float().mean().item()
    return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()


results = trainer.evaluate()
print("Wyniki ewaluacji:", results)


trainer.save_model("./fine_tuned_bert")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Michał\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Michał\AppData\Local\Temp\ipykernel_17268\1775251010.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  2%|▏         | 12/500 [00:01<00:51,  9.40it/s]

{'loss': 1.6264, 'grad_norm': 6.784598350524902, 'learning_rate': 1.9600000000000002e-05, 'epoch': 0.02}


  4%|▍         | 22/500 [00:02<00:47, 10.15it/s]

{'loss': 1.3162, 'grad_norm': 6.381491184234619, 'learning_rate': 1.9200000000000003e-05, 'epoch': 0.04}


  6%|▋         | 32/500 [00:03<00:45, 10.19it/s]

...

{'eval_loss': 0.7409616112709045, 'eval_accuracy': 0.7065000534057617, 'eval_runtime': 3.4255, 'eval_samples_per_second': 583.858, 'eval_steps_per_second': 36.491, 'epoch': 1.0}


100%|██████████| 500/500 [00:54<00:00,  9.20it/s]


{'train_runtime': 54.3316, 'train_samples_per_second': 147.244, 'train_steps_per_second': 9.203, 'train_loss': 0.902793779373169, 'epoch': 1.0}


100%|██████████| 125/125 [00:03<00:00, 36.69it/s]


Wyniki ewaluacji: {'eval_loss': 0.7409616112709045, 'eval_accuracy': 0.7065000534057617, 'eval_runtime': 3.4379, 'eval_samples_per_second': 581.748, 'eval_steps_per_second': 36.359, 'epoch': 1.0}


Wywołać tylko raz by pobrać potem będą problemy bo model już jest załadowany

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Pobranie modelu i tokenizatora
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Zapisanie modelu i tokenizatora lokalnie
model.save_pretrained("./twitter-roberta-sentiment")
tokenizer.save_pretrained("./twitter-roberta-sentiment")
print("Model i tokenizator zapisane lokalnie.")


Model i tokenizator zapisane lokalnie.


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

size = 20
local_model_path = "./twitter-roberta-sentiment"
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForSequenceClassification.from_pretrained(local_model_path)

inputs = tokenizer(list(sample.head(size).text), padding=True, truncation=True, return_tensors="pt", max_length=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}

# Predykcja
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

label_map = {0: "negative", 1: "neutral", 2: "positive"}

mapped_predictions = [label_map[label] for label in predictions.tolist()]

results = [[pred, val] for pred, val in zip(mapped_predictions, list(sample.head(size).rating))]
print(results)

[['positive', 5.0], ['negative', 4.0], ['positive', 5.0], ['negative', 4.0], ['negative', 3.0], ['positive', 5.0], ['positive', 5.0], ['positive', 5.0], ['neutral', 3.0], ['positive', 5.0], ['negative', 3.0], ['positive', 5.0], ['positive', 5.0], ['positive', 5.0], ['positive', 5.0], ['positive', 5.0], ['positive', 5.0], ['negative', 3.0], ['positive', 5.0], ['positive', 5.0]]


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = "./fine_tuned_bert"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

inputs = tokenizer(list(sample.head(20).text), padding=True, truncation=True, return_tensors="pt", max_length=128)

# Przeniesienie danych na GPU (jeśli dostępne)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {key: val.to(device) for key, val in inputs.items()}


model.eval()  
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)


print("Wartości:", [int(x) for x in sample.head(20).rating])
print("Predykcje:", [x+1 for x in predictions.tolist()])


Wartości: [5, 4, 5, 4, 3, 5, 5, 5, 3, 5, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5]
Predykcje: [5, 3, 5, 2, 3, 5, 5, 5, 3, 5, 2, 5, 5, 5, 4, 5, 5, 3, 5, 5]
